# 1. Imports

In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np


# 2. Importing the Datasets

In [2]:
ddos_portmap_2018_df = pd.read_csv("../data/train_test/CSE_CIC_IDS2018/ddos_portmap_2018_train.csv", index_col=0)

In [3]:
ddos_ldap_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_ldap_2019_train.csv", index_col=0)

In [4]:
ddos_netbios_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_netbios_2019_train.csv", index_col=0)

In [5]:
ddos_syn_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_syn_2019_train.csv", index_col=0)

In [6]:
ddos_udp_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_udp_2019_train.csv", index_col=0)

In [7]:
target_feature = 'Label'

# 3. Feature Selection Method

In [8]:
'''from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def forward_selection(X, y, features, metric='accuracy'):
    selected_features = []
    selected_scores = []

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    while len(selected_features) < len(features):
        remaining_features = list(set(features) - set(selected_features))

        if not remaining_features:
            break  # Exit the loop if there are no more features to evaluate

        best_score = 0
        best_feature = None

        for feature in remaining_features:
            current_features = selected_features + [feature]

            model = RandomForestClassifier(random_state=42)
            model.fit(X_train[current_features], y_train)

            y_pred = model.predict(X_test[current_features])

            if metric == 'accuracy':
                score = accuracy_score(y_test, y_pred)
                print(f"Feature: {feature}, Score: {score}")

            if score > best_score:
                best_score = score
                best_feature = feature

        selected_features.append(best_feature)
        selected_scores.append(best_score)
        print(f"Selected Features: {selected_features}")

    return selected_features, selected_scores'''




In [15]:
'''def stable_ForwardSelection(train_df, num_splits, verbose):
    selected_features = []

    df_split = np.array_split(train_df, num_splits)

    for small_df in df_split:
        X_train = small_df.drop(['Label'], axis=1)
        y_train = small_df['Label']


        features, _ = forward_selection(X_train, y_train, X_train.columns)
        selected_features += features

    # Use forward selection on the entire training dataset
    forward_selected_features, _ = forward_selection(train_df.drop(['Label'], axis=1), train_df['Label'], train_df.columns)

    # Extract the selected features from the tuple returned by forward_selection
    selected_features += forward_selected_features

    rank_data = {'feature': train_df.columns.tolist(),
                 'forward_selection_rank': [selected_features.count(feature) for feature in train_df.columns]}

    rank_df = pd.DataFrame(rank_data)

    if verbose:
        print(rank_df)

    return rank_df
'''



In [10]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

def rfe_selection(X_train, y_train, features):
    lreg = LinearRegression()
    
    # Create an RFE object for feature selection
    rfe = RFE(lreg, n_features_to_select=1)
    
    # Fit the RFE to the training data
    rfe.fit(X_train, y_train)
    
    # Get the selected feature ranking
    feature_ranking = rfe.ranking_

    # Get the selected feature names based on their ranking
    selected_features = list(features[np.argsort(feature_ranking)])
    
    return selected_features

In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

def stable_RFE(train_df, num_splits, verbose):
    selected_features = []

    df_split = np.array_split(train_df, num_splits)

    for small_df in df_split:
        X_train = small_df.drop(['Label'], axis=1)
        y_train = small_df['Label']

        features = rfe_selection(X_train, y_train, X_train.columns)
        selected_features += features

    # Use RFE on the entire training dataset
    rfe_selected_features = rfe_selection(train_df.drop(['Label'], axis=1), train_df['Label'], train_df.columns)

    # Filter out 'Label' from the selected features
    selected_features = [feature for feature in selected_features if feature != 'Label']

    # Normalize the ranks between 0 and 1
    max_rank = len(selected_features)
    normalized_ranks = [1 - (selected_features.index(feature) / max_rank) for feature in train_df.columns if feature != 'Label']

    rank_data = {'feature': [feature for feature in train_df.columns if feature != 'Label'],
                 'rfe_selection_rank': normalized_ranks}

    rank_df = pd.DataFrame(rank_data)

    if verbose:
        print(rank_df)

    return rank_df







In [18]:
portmap_ranking = stable_RFE(ddos_portmap_2018_df, 10, verbose=False)

In [19]:
portmap_ranking.head(20)

,feature,rfe_selection_rank
0,DestinationPort,0.939744
1,FlowDuration,0.982051
2,TotalFwdPackets,0.996154
3,TotalBackwardPackets,1.000000
4,TotalLengthofFwdPackets,0.985897
5,TotalLengthofBwdPackets,0.994872
6,FwdPacketLengthMax,0.967949
7,FwdPacketLengthMin,0.941026
8,FwdPacketLengthMean,0.946154
9,FwdPacketLengthStd,0.944872


# 4. Saving the Ranking

CODE TO SAVE A CSV FILE WITH:
- 'feature' is a column of all of the features in the original dataset
- 'ranking' is a column with the rankings (in range 0 to 1) of the feature

In [20]:
portmap_ranking = stable_RFE(ddos_portmap_2018_df, 10, verbose=False)
portmap_ranking.to_csv("../ranking/CSE_CIC_IDS2018/ddos_portmap_2018_RFE.csv")
portmap_ranking.head(20)

,feature,rfe_selection_rank
0,DestinationPort,0.939744
1,FlowDuration,0.982051
2,TotalFwdPackets,0.996154
3,TotalBackwardPackets,1.000000
4,TotalLengthofFwdPackets,0.985897
5,TotalLengthofBwdPackets,0.994872
6,FwdPacketLengthMax,0.967949
7,FwdPacketLengthMin,0.941026
8,FwdPacketLengthMean,0.946154
9,FwdPacketLengthStd,0.944872


In [21]:
ldap_ranking = stable_RFE(ddos_ldap_2019_df, 10, verbose=False)
ldap_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_ldap_2019_RFE.csv")
ldap_ranking.head(20)

,feature,rfe_selection_rank
0,FlowID,0.917647
1,SourceIP,0.968235
2,SourcePort,0.945882
3,DestinationIP,0.941176
4,DestinationPort,0.914118
5,Protocol,0.950588
6,FlowDuration,0.964706
7,TotalFwdPackets,0.988235
8,TotalBackwardPackets,0.991765
9,TotalLengthofFwdPackets,0.994118


In [22]:
netbios_ranking = stable_RFE(ddos_netbios_2019_df, 10, verbose=False)
netbios_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_netbios_2019_RFE.csv")
netbios_ranking.head(20)

,feature,rfe_selection_rank
0,FlowID,0.917647
1,SourceIP,0.923529
2,SourcePort,0.918824
3,DestinationIP,0.921176
4,DestinationPort,0.915294
5,Protocol,0.947059
6,FlowDuration,0.943529
7,TotalFwdPackets,0.965882
8,TotalBackwardPackets,0.994118
9,TotalLengthofFwdPackets,0.983529


In [23]:
syn_ranking = stable_RFE(ddos_syn_2019_df, 10, verbose=False)
syn_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_syn_2019_RFE.csv")
syn_ranking.head(20)

,feature,rfe_selection_rank
0,FlowID,0.909412
1,SourceIP,0.945882
2,SourcePort,0.908235
3,DestinationIP,0.947059
4,DestinationPort,0.910588
5,Protocol,0.963529
6,FlowDuration,0.943529
7,TotalFwdPackets,0.989412
8,TotalBackwardPackets,0.996471
9,TotalLengthofFwdPackets,0.991765


In [24]:
udp_ranking = stable_RFE(ddos_udp_2019_df, 10, verbose=False)
udp_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_udp_2019_RFE.csv")
udp_ranking.head(20)

,feature,rfe_selection_rank
0,FlowID,0.921176
1,SourceIP,0.943529
2,SourcePort,0.922353
3,DestinationIP,0.931765
4,DestinationPort,0.918824
5,Protocol,0.950588
6,FlowDuration,0.968235
7,TotalFwdPackets,0.977647
8,TotalBackwardPackets,0.978824
9,TotalLengthofFwdPackets,0.962353
